In [1]:
!nvidia-smi -L

# colab resource monitor
from urllib.request import urlopen
exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
_colabMonitor = ColabMonitor().start()

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-613c0d86-2d1f-e60e-b323-e40d5ed87ba1)
Now live at : http://colab-monitor.smankusors.com/60d9d54a67a9f


In [2]:
!pip install transformers
!pip install datasets
!pip install rouge-score
!pip nltk

from IPython.display import clear_output
clear_output()

# Fine-tuning a model on a summarization task
This notebook will show how to fine-tune one of the 🤗 Transformers model to a **summarization task**. We will use the **XSum dataset** (for extreme summarization) which contains BBC articles accompanied with single-sentence summaries.<br>
- [XSum from huggingface](https://huggingface.co/datasets/xsum)

In [3]:
# set parameters for model
model_checkpoint = "t5-small"

# set parameters for training
batch_size = 16
epochs = 1


# check execution time for whole code
import time
s_time = time.time()

# import packages
import datasets

import pandas as pd
import numpy as np

import random
import collections
import tqdm
import nltk
nltk.download('punkt')

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from transformers import default_data_collator

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# datasets : 1.8.0  |  pd : 1.1.5  |  np : 1.19.5  |  tqdm : 4.41.1  |  nltk : 3.2.5  |  transformers : 4.8.1  |  torch : 1.9.0+cu102
print(f'datasets : {datasets.__version__}  |  pd : {pd.__version__}  |  np : {np.__version__}  |  tqdm : {tqdm.__version__}  |  nltk : {nltk.__version__}  |  transformers : {transformers.__version__}  |  torch : {torch.__version__}')
print('device :', device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
datasets : 1.8.0  |  pd : 1.1.5  |  np : 1.19.5  |  tqdm : 4.41.1  |  nltk : 3.2.5  |  transformers : 4.8.1  |  torch : 1.9.0+cu102
device : cuda


In [4]:
############### Prepare Data ###############

# load dataset & metric
dset_dict = datasets.load_dataset('xsum')
metric = datasets.load_metric('rouge')

# # check dataset
# print('\n>>> dataset object :')
# display(dset_dict)
# print('\n>>> sample data :')
# display(dset_dict['train'][0])


# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


# add prefix when using t5 model
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""


# preprocess text (tokenizer)
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix+doc for doc in examples['document']]
    model_inputs = tokenizer(inputs,  max_length=max_input_length, truncation=True)

    # setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=max_target_length, truncation=True)
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


tokenized_datasets = dset_dict.map(preprocess_function, batched=True)  # batched=True -> use multi-threading to encode texts by batches together
# tokenized_datasets
print('\n>>> Successfully prepared data!\n')

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)
Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-05c14d4598578078.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-0c1a02dd0f880d0e.arrow




>>> Successfully prepared data!



In [4]:
############### Fine-tuning the model ###############

In [5]:
# check execution time for whole code
e_time = time.time()
time_elapsed = e_time - s_time
print(f'Total time elapsed : {int(time_elapsed//60)} min {int(time_elapsed%60)} sec')

Total time elapsed : 0 min 15 sec
